In [2]:
import boto3
import sagemaker
import sagemaker.session

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"ToyookaModelPackageGroupName"

In [ ]:
#ここからはdataset（batchじゃない）

In [ ]:
!mkdir -p data_toyooka
#local_path = "data_toyooka/abalone-dataset.csv"

#s3 = boto3.resource("s3")
#s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
#    "dataset/abalone-dataset.csv",
#    local_path
#)

#base_uri = f"s3://{default_bucket}/abalone"
#input_data_uri = sagemaker.s3.S3Uploader.upload(
#    local_path=local_path, 
#    desired_s3_uri=base_uri,
#)
#print(input_data_uri)

In [3]:
local_path = "data_toyooka/toyooka_20180807_20210331.csv"
print(local_path)

data_toyooka/toyooka_20180807_20210331.csv


In [4]:
base_uri = f"s3://{default_bucket}/toyooka"
print(base_uri)

s3://sagemaker-ap-northeast-1-899528287347/toyooka


In [5]:
import pandas as pd
df = pd.read_csv("s3://plantpathology/toyooka_20180807_20210331.csv")
df.to_csv('toyooka_20180807_20210331.csv')

In [6]:
import os
cwd = os.getcwd()
print(cwd)

/root/aaa/toyooka


In [7]:
#csvをdata_toyookaへ移動するコード（コピーななくて切取して移動）
import shutil
new_path = shutil.move('/root/aaa/toyooka/toyooka_20180807_20210331.csv', '/root/aaa/toyooka/data_toyooka')

In [8]:
#s3 = boto3.resource("s3")
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path, 
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-ap-northeast-1-899528287347/toyooka/toyooka_20180807_20210331.csv


In [ ]:
#ここからはbatch_dataset

In [ ]:
#local_path = "data_toyooka/toyooka-dataset-batch"

#s3 = boto3.resource("s3")
#s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
#    "dataset/abalone-dataset-batch",
#    local_path
#)

#base_uri = f"s3://{default_bucket}/abalone"
#batch_data_uri = sagemaker.s3.S3Uploader.upload(
#    local_path=local_path, 
#    desired_s3_uri=base_uri,
#)
#print(batch_data_uri)

In [9]:
local_path = "data_toyooka/toyooka-batch_20180807_20210331.csv"
print(local_path)

data_toyooka/toyooka-batch_20180807_20210331.csv


In [10]:
base_uri = f"s3://{default_bucket}/toyooka"
print(base_uri)

s3://sagemaker-ap-northeast-1-899528287347/toyooka


In [11]:
df.to_csv('toyooka-batch_20180807_20210331.csv')

In [12]:
import shutil
new_path = shutil.move('/root/aaa/toyooka/toyooka-batch_20180807_20210331.csv', '/root/aaa/toyooka/data_toyooka')

In [13]:
batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path, 
    desired_s3_uri=base_uri,
)
print(batch_data_uri)

s3://sagemaker-ap-northeast-1-899528287347/toyooka/toyooka-batch_20180807_20210331.csv


In [14]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri,
)

In [ ]:
!mkdir -p toyooka

In [15]:
%%writefile toyooka/preprocessing.py
import argparse
import os
import requests
import tempfile
import numpy as np
import pandas as pd


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#ヘッダー追加してる
# Because this is a headerless CSV file, specify the column names here.
#feature_columns_names = [
#    "sex",
#    "length",
#    "diameter",
#    "height",
#    "whole_weight",
#    "shucked_weight",
#    "viscera_weight",
#    "shell_weight",
#]
#label_column = "rings"

#feature_columns_dtype = {
#    "sex": str,
#    "length": np.float64,
#    "diameter": np.float64,
#    "height": np.float64,
#    "whole_weight": np.float64,
#    "shucked_weight": np.float64,
#    "viscera_weight": np.float64,
#    "shell_weight": np.float64
#}
#label_column_dtype = {"rings": np.float64}

#不明だが、train, valid分割するのに必要そうだから残す
def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)
    return z


if __name__ == "__main__":
    base_dir = "/opt/ml/processing"

    df = pd.read_csv("s3://plantpathology/toyooka_20180807_20210331.csv")
#    df = pd.read_csv(
#        f"{base_dir}/input/abalone-dataset.csv",
#        header=None, 
#        names=feature_columns_names + [label_column],
#        dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype)
#    )

#    numeric_features = list(feature_columns_names)
#    numeric_features.remove("sex")
#ScikitlearnのPipelineライブラリ（今はスケーリングしないから使わない）
#    numeric_transformer = Pipeline(
#        steps=[
#            ("imputer", SimpleImputer(strategy="median")),
#            ("scaler", StandardScaler())
#        ]
#    )
#性別をクラス分けするためワンホットエンコード
#    categorical_features = ["sex"]
#    categorical_transformer = Pipeline(
#        steps=[
#            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
#            ("onehot", OneHotEncoder(handle_unknown="ignore"))
#        ]
#    )
#コラム列をまるごと整形する
#    preprocess = ColumnTransformer(
#        transformers=[
#            ("num", numeric_transformer, numeric_features),
#            ("cat", categorical_transformer, categorical_features)
#        ]
#    )
    
#    y = df.pop("rings")
#    X_pre = preprocess.fit_transform(df)
#    y_pre = y.to_numpy().reshape(len(y), 1)
    
#    X = np.concatenate((y_pre, X_pre), axis=1)
#前のコードでエックスを使ってるから、そのまま使う X = dfは追加した
    X = df
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7*len(X)), int(.85*len(X))])

    
    pd.DataFrame(train).to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(f"{base_dir}/validation/validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(f"{base_dir}/test/test.csv", header=False, index=False)

Overwriting toyooka/preprocessing.py


In [17]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type="ml.m5.xlarge",
    instance_count=processing_instance_count,
    base_job_name="sklearn-toyooka-process",
    role=role,
)

In [18]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name="ToyookaProcess",
    processor=sklearn_processor,
    inputs=[
      ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test")
    ],
    code="toyooka/preprocessing.py",
)

In [19]:
model_path = f"s3://{default_bucket}/ToyookaTrain"

In [20]:
from sagemaker.estimator import Estimator


image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.xlarge"
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=model_path,
    role=role,
)
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=60,
    max_depth=3,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0
)

In [21]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="ToyookaTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
)

In [22]:
%%writefile toyooka/evaluation.py
import json
import pathlib
import pickle
import tarfile
import joblib
import numpy as np
import pandas as pd
import xgboost


from sklearn.metrics import mean_squared_error


if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    
    model = pickle.load(open("xgboost-model", "rb"))

    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)
    
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    
    X_test = xgboost.DMatrix(df.values)
    
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    std = np.std(y_test - predictions)
    report_dict = {
        "regression_metrics": {
            "mse": {
                "value": mse,
                "standard_deviation": std
            },
        },
    }

    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting toyooka/evaluation.py


In [23]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="script-toyooka-eval",
    role=role,
)

In [24]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)
step_eval = ProcessingStep(
    name="ToyookaEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="toyooka/evaluation.py",
    property_files=[evaluation_report],
)

In [25]:
from sagemaker.model import Model


model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [26]:
from sagemaker.inputs import CreateModelInput


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.eia1.medium",
)

In [27]:
from sagemaker.workflow.steps import CreateModelStep


step_create_model = CreateModelStep(
    name="ToyookaCreateModel",
    model=model,
    inputs=inputs,
)

In [28]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=f"s3://{default_bucket}/ToyookaTransform"
)

In [29]:
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="ToyookaTransform",
    transformer=transformer,
    inputs=TransformInput(data=batch_data)
)

In [30]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.step_collections import RegisterModel


model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)
step_register = RegisterModel(
    name="ToyookaRegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics
)

In [31]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.mse.value"
    ),
    right=6.0
)

In [32]:
step_cond = ConditionStep(
    name="ToyookaMSECond",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[], 
)

In [33]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"ToyookaPipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        model_approval_status,
        input_data,
        batch_data,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

In [34]:
import json

json.loads(pipeline.definition())

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-1-899528287347/toyooka/toyooka_20180807_20210331.csv'},
  {'Name': 'BatchData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-ap-northeast-1-899528287347/toyooka/toyooka-batch_20180807_20210331.csv'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'ToyookaProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.m5.xlarge',
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '354813040037.dkr.ecr.a

In [35]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-1:899528287347:pipeline/toyookapipeline',
 'ResponseMetadata': {'RequestId': '03bd69db-8fea-4eb6-99d0-2b30114253f1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '03bd69db-8fea-4eb6-99d0-2b30114253f1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Tue, 27 Dec 2022 04:46:07 GMT'},
  'RetryAttempts': 0}}

In [36]:
execution = pipeline.start()

In [37]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-1:899528287347:pipeline/toyookapipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-1:899528287347:pipeline/toyookapipeline/execution/7a7vwyz09khj',
 'PipelineExecutionDisplayName': 'execution-1672116371936',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'toyookapipeline',
  'TrialName': '7a7vwyz09khj'},
 'CreationTime': datetime.datetime(2022, 12, 27, 4, 46, 11, 806000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 12, 27, 4, 46, 11, 806000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-northeast-1:899528287347:user-profile/d-qebiusz2grwq/sagemakertejima',
  'UserProfileName': 'sagemakertejima',
  'DomainId': 'd-qebiusz2grwq'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:ap-northeast-1:899528287347:user-profile/d-qebiusz2grwq/sagemakertejima',
  'UserProfileName': 'sagemakertejima',
  'DomainId': 'd-qebiusz2grwq'},
 'Respo

In [38]:
execution.wait()

WaiterError: Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"

In [39]:
execution.list_steps()

[{'StepName': 'ToyookaProcess',
  'StartTime': datetime.datetime(2022, 12, 27, 4, 46, 14, 9000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 12, 27, 4, 50, 38, 397000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'ClientError: AlgorithmError: See job logs for more information',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-1:899528287347:processing-job/pipelines-7a7vwyz09khj-toyookaprocess-vdjlt3rglx'}},
  'AttemptCount': 0}]

In [ ]:
evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
json.loads(evaluation_json)

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()

In [ ]:
evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
json.loads(evaluation_json)

In [ ]:
#execution.stop()

In [ ]:
#pipeline.delete()